In [1]:
import pyltp
import jieba

In [52]:
import os
LTP_DATA_DIR = '/Users/hang.xiang/Downloads/ltp_data_v3.4.0'  # ltp模型目录的路径
par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型路径，模型名称为`parser.model`

from pyltp import Parser
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import SentenceSplitter

segmentor = Segmentor()  # 初始化实例
segmentor.load(LTP_DATA_DIR+'/cws.model')
postagger = Postagger() # 初始化实例
postagger.load(LTP_DATA_DIR+'/pos.model')
parser = Parser() # 初始化实例
parser.load(LTP_DATA_DIR+'/parser.model')  # 加载模型


sentence = '华泰证券表示，在纳入MSCI指数之后，A股将迎来约850亿元人民币的增量资金。'
words = list(segmentor.segment(sentence))
# print(type(words))
# words = ['元芳', '你', '怎么', '看']
# postags = ['nh', 'r', 'r', 'v']
postags = postagger.postag(words)
arcs = parser.parse(words, postags)  # 句法分析

print("\t".join("%d:%s:%s" % (arc.head, arc.relation,w) for arc, w in zip(arcs,words)))

segmentor.release()
postagger.release()
parser.release()  # 释放模型

2:ATT:华泰	3:SBV:证券	0:HED:表示	3:WP:，	14:ADV:在	9:ATT:纳入	8:ATT:MSCI	6:VOB:指数	5:POB:之后	5:WP:，	12:ATT:A	14:SBV:股	14:ADV:将	3:VOB:迎来	16:ATT:约	17:ATT:850亿	18:ATT:元	21:ATT:人民币	18:RAD:的	21:ATT:增量	14:VOB:资金	3:WP:。


In [28]:
sents = SentenceSplitter.split('从国际经验来看，纳入MSCI指数后，相关股市的投资者风格将更加稳健，更加偏好业绩稳定、流动性好、风险低的优质蓝筹股。')
print(list(sents))

['从国际经验来看，纳入MSCI指数后，相关股市的投资者风格将更加稳健，更加偏好业绩稳定、流动性好、风险低的优质蓝筹股。']


In [49]:
class Whotalk:
    def __init__(self, talkwordnum=60):
        # init cut module
        self.segmentor = Segmentor()
        self.segmentor.load(LTP_DATA_DIR+'/cws.model')
        # init postag module
        self.postagger = Postagger()
        self.postagger.load(LTP_DATA_DIR+'/pos.model')
        # init dependency parsing module
        self.parser = Parser()
        self.parser.load(LTP_DATA_DIR+'/parser.model')
        # init talk words
        with open('saywords.json', 'r') as f:
            self.talkwords = [word  for word, score in json.load(f)[:60]]

        print('Whotalk init complete.')
    
    def istalkindoc(self, doc):
        for talk_w in self.talkwords:
            if talk_w in doc:
                return True
        return False
    
    def checkwhotalk(self, doc):
        ret = {}
        sentences = SentenceSplitter.split(doc)
        for sentence in sentences:
            if not any([w in sentence for w in self.talkwords]): 
                continue
            # find all talk words
            words = list(self.segmentor.segment(sentence))
            # for each talk word, get the whole sentense
            # for those sentense, get the postag
            postags = self.postagger.postag(words)
            # according to postag get the who and opinion
            arcs = self.parser.parse(words, postags)
            
            for i, (arc, word) in enumerate(zip(arcs, words)):
                if arc.head == 0 and word in self.talkwords:
                    talkindex = i
            
            who = None, 100
            for word, arc in list(zip(words, arcs))[:i]:
                if arc.relation == 'SBV' and who[1] > arc.head:
                    who = word, arc.head
            ret[who[0]] = ''.join(words[talkindex+1:])
            
        return ret
    
    def release(self):
        # release cut module
        self.segmentor.release()
        # release postag module
        self.postagger.release()
        # release dp module
        self.parser.release()

In [51]:
wt = Whotalk()
ret = wt.checkwhotalk('''
\u3000\u30006月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。
22日，在222只纳入MSCI指数的A股股票中，银行股全线飘红，其中招商银行领涨，涨幅达6.66%。
保险股和券商股的表现也可圈可点。在这222只股票中，金融板块的股票数量和总市值占比均位居首位。
分析人士指出，银行股股息率高、估值低、收益稳定，对于资金量较大、投资期限较长的资金存在相当大的吸引力。
从国际经验来看，纳入MSCI指数后，相关股市的投资者风格将更加稳健，更加偏好业绩稳定、流动性好、风险低的优质蓝筹股。
\n\u3000\u3000银行股具估值优势\n\u3000\u30006月22日，A股金融股表现强势，板块涨幅达1.19%。
据平安证券统计，在222只纳入MSCI指数的A股股票中，金融板块市值占比达41.32%，其中银行、证券和保险子板块的市值占比分别为27.30%、7.75%和6.24%。
\n\u3000\u3000多家券商研究报告认为，银行等金融股受到纳入MSCI指数的提振，未来更多境外资金将进入金融股。
\n\u3000\u3000华泰证券表示，在纳入MSCI指数之后，A股将迎来约850亿元人民币的增量资金。
其中，考虑到222只股票中，金融股市值占比约42%，并且金融股体量大、流动性强、估值低，符合境外资金偏好，有望迎来超过权重的资金比例。
预计金融股有望吸引增量资金约350亿元人民币。\n\u3000\u3000新富资本证券投资中心研究总监廖云龙认为，银行股脱颖而出的主要原因是低估值。
市场预期银行股基本面改善，同时市场整体风格偏保守，偏爱低估值。22日银行股的大涨是市场风格的延续，是投资者对龙头股和对低估值的防御性选择。
\n\u3000\u3000东方财富choice统计的一季度末QFII重仓股数据显示，银行股是QFII关注的重点之一。在前五十大重仓股中，北京银行、南京银行、宁波银行、上海银行榜上有名。\n
''')
# print(wt.talkwords)
print(ret)
wt.release()

Whotalk init complete.
{'人士': '，银行股股息率高、估值低、收益稳定，对于资金量较大、投资期限较长的资金存在相当大的吸引力。', '报告': '，银行等金融股受到纳入MSCI指数的提振，未来更多境外资金将进入金融股。', '证券': '，在纳入MSCI指数之后，A股将迎来约850亿元人民币的增量资金。', '廖云龙': '，银行股脱颖而出的主要原因是低估值。'}
